## Source of inspiration

[HF news of the week early october](/posts/2022-10-01-logbook-October-22.html#friday-107)

There is a new version of [Gradio](https://github.com/gradio-app/gradio/releases/tag/v3.4) announced. Have to try it.

These apps can then be hosted on huggingface. Let's try that.


# How to do it